In [ ]:
using CSV, DataFrames             # Pour charger et organiser les données
using Gadfly                      # Pour générer des graphiques
using Distributions               # Pour utiliser les distributions statistiques
using GLM                         # Libriairie pour la régression et les modèles linéaires généralisés
using LinearAlgebra               # Pour utiliser les fonctions d'algèbre linéaire
using MLBase                      # Pour utiliser les différents scores et la courbe ROC

In [ ]:
"""
    auc(gt::Array{<:Real}, scores::Array{<:Real})

    Compute the area under the ROC curve based on the ground truth `gt` and the success probability `scores`.

    See also `roc()` of MLBase.
"""
function auc(gt::Array{<:Real},scores::Array{<:Real})

    # Compute the ROC curve for 100 equally spaced thresholds - see `roc()`
    r = roc(gt, scores)

    # Compute the true positive rate and false positive rate
    tpr = true_positive_rate.(r)
    fpr = false_positive_rate.(r)

    # Numerical computation of the area under the ROC curve
    p = sortperm(fpr)

    permute!(tpr,p)
    permute!(fpr,p)

    area = 0.0

    for i in 2:length(tpr)
        dx = fpr[i] - fpr[i-1]
        dy = tpr[i] - tpr[i-1]
        area += dx*tpr[i-1] + dx*dy/2
    end

    return area

end


"""
    rocplot(gt::Array{<:Real},scores::Array{<:Real})

    Show the ROC curve corresponding to the ground truth `gt` and the success probability `scores`.

    The curve is computed for 100 equally spaced thresholds.
"""
function rocplot(gt::Array{<:Real},scores::Array{<:Real})

    # Compute the ROC curve for 100 equally spaced thresholds - see `roc()`
    r = roc(gt, scores, 0:.01:1)

    # Compute the true positive rate and false positive rate
    tpr = true_positive_rate.(r)
    fpr = false_positive_rate.(r)

    return plot(x=fpr, y=tpr, Geom.line, Geom.abline(color="red", style=:dash),
        Guide.xlabel("False Positive Rate"), Guide.ylabel("True Positive Rate"))

end

# Chargement des données

Le jeu de données est partionné en deux sous-ensembles : un ensemble d'entraînement et un ensemble de test. L'ensemble d'entraînement, comportant les informations de 455 patients, sert à estimer les paramètres de notre modèle.

In [ ]:
# Chargement l'ensemble d'entraînement

data = CSV.read("train.csv")
first(data, 5)

# 3.2 Régression logistique

Nous avons déterminés lors de l'analyse de composantes/l'analyse de multicolinéarité que nous avons fait dans un autre document quelles variables il fallait utiliser pour minimiser les problèmes de multicolinéarité et quelles variables conserver. On utilise donc les variables: "radius", "texture", "smoothness", "compactness", "concavity" et "symmetry" comme variable explicative dans le modèle linéaire généralisé.

## Estimation par maximum de la vraisemblance du modèle de régression logistique

In [ ]:
function trainGlmWith(df, formula=@formula(diagnosis ~ radius + texture + smoothness + compactness + concavity + symmetry))
    return glm(formula, df,  Bernoulli(), LogitLink())
end
# M = trainGlmWith(data)

In [ ]:
# Pour chacune des observations, estimation de la probabilité de survie
# θ̂ = predict(M)

## K cross validation

In [ ]:
# K-cross validation
function findAllIndexes(length, nb_blocks)
    return [(convert(Int16, floor((i-1)*length/nb_blocks))+1, convert(Int16, floor(i*length/nb_blocks))) for i = 1:nb_blocks]
end

In [ ]:
function glm_split_train_test(df, index)
    train = df[Not(index[1]:index[2]), :]
    test = df[index[1]:index[2], :]
    return train, test
end

In [ ]:
function glm_kcross(df, k) 
    indexes = findAllIndexes(size(df, 1), k)
    θ̂  = Float64[]
    for interval = indexes
        train, test = glm_split_train_test(df, interval)
        M = trainGlmWith(train)
        θ̂ᵢ = predict(M, test)
        append!(θ̂, θ̂ᵢ)
    end
    return θ̂
end
θ̂ = glm_kcross(data, 15)

# Mesure de la qualité du modèle

Supposons que l'on prédit que la tumeur est maligne si θ̂ > 1/2.

In [ ]:
# Le taux de bonnes prédictions
ŷ = zeros(Int64,length(data.diagnosis))
ŷ[θ̂ .> 1/2] .= 1 

println("Le taux de bonnes prédictions est de ", round(correctrate(data.diagnosis, ŷ), digits=3),".")


In [ ]:
# Calcul du taux de vrais positifs et de faux positifs pour un seuil de 1/2.

r = roc(data.diagnosis, θ̂, 1/2)

println("La sensibilité est de ", round(recall(r), digits=3))
println("La spécificité est de ", round(precision(r), digits=3))
println("Le score F1 est de ", round(f1score(r), digits=3))

In [ ]:
# Calcul de l'aire sous la courbe ROC

A = auc(data.diagnosis, θ̂)

In [ ]:
# Affichage de la courbe ROC

rocplot(data.diagnosis, θ̂)